In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, roc_curve, auc
from keras.models import Sequential
from keras.layers import Dense, Dropout
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier
from imblearn.under_sampling import RandomUnderSampler

# Load the dataset
data = pd.read_csv("/kaggle/input/fraud-analytics/creditcard.csv")

# Data preprocessing
X = data.drop("Class", axis=1)
y = data["Class"]

# Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Undersample the majority class
undersampler = RandomUnderSampler(sampling_strategy=0.5, random_state=42)
X_train_resampled, y_train_resampled = undersampler.fit_resample(X_train, y_train)

# Autoencoder architecture
input_dim = X_train_resampled.shape[1]
encoding_dim = 16

autoencoder = Sequential([
    Dense(32, input_shape=(input_dim,), activation='relu'),
    Dropout(0.1),
    Dense(16, activation='relu'),
    Dropout(0.1),
    Dense(8, activation='relu'),
    Dense(16, activation='relu'),
    Dropout(0.1),
    Dense(32, activation='relu'),
    Dense(input_dim, activation='linear')
])

# Compile the autoencoder
autoencoder.compile(optimizer='adam', loss='mse')

# Train the autoencoder
history = autoencoder.fit(X_train_resampled, X_train_resampled, epochs=25, batch_size=64, shuffle=True, validation_data=(X_test, X_test))

# Plot the training loss
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Autoencoder Training')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Reconstruct the test data
reconstructed_data = autoencoder.predict(X_test)

# Calculate reconstruction errors
mse = np.mean(np.power(X_test - reconstructed_data, 2), axis=1)

# Threshold optimization
fpr, tpr, thresholds = roc_curve(y_test, mse)
roc_auc = auc(fpr, tpr)
optimal_idx = np.argmax(tpr - fpr)
optimal_threshold = thresholds[optimal_idx]

# Identify potential anomalies
anomalies = X_test[mse > optimal_threshold]

# Ensemble methods
clf1 = LogisticRegression()
clf2 = RandomForestClassifier()
clf3 = SVC(kernel='linear', probability=True)

ensemble_clf = VotingClassifier(estimators=[('lr', clf1), ('rf', clf2), ('svc', clf3)], voting='soft')
ensemble_clf.fit(X_train_resampled, y_train_resampled)

# Evaluate the performance of the ensemble classifier
y_pred_ensemble = ensemble_clf.predict(X_test)

precision = precision_score(y_test, y_pred_ensemble)
recall = recall_score(y_test, y_pred_ensemble)
f1 = f1_score(y_test, y_pred_ensemble)
accuracy = accuracy_score(y_test, y_pred_ensemble)

print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)
print("Accuracy:", accuracy)

In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, roc_curve, auc
from keras.models import Model
from keras.layers import Input, Dense, Dropout, Lambda, Layer
import tensorflow.keras.backend as K
import tensorflow as tf
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier
from imblearn.under_sampling import RandomUnderSampler

# Load the dataset
data = pd.read_csv("/kaggle/input/fraud-analytics/creditcard.csv")

# Data preprocessing
X = data.drop("Class", axis=1)
y = data["Class"]

# Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, stratify=y, random_state=42)

# Undersample the majority class
undersampler = RandomUnderSampler(sampling_strategy=0.5, random_state=42)
X_train_resampled, y_train_resampled = undersampler.fit_resample(X_train, y_train)

# VAE architecture
input_dim = X_train_resampled.shape[1]
encoding_dim = 16

# Encoder
inputs = Input(shape=(input_dim,))
x = Dense(32, activation='relu')(inputs)
x = Dropout(0.1)(x)
x = Dense(16, activation='relu')(x)
z_mean = Dense(encoding_dim)(x)
z_log_var = Dense(encoding_dim)(x)

# Sampling from the latent distribution
def sampling(args):
    z_mean, z_log_var = args
    epsilon = tf.random.normal(shape=(K.shape(z_mean)[0], encoding_dim), mean=0., stddev=1.0)
    return z_mean + K.exp(z_log_var / 2) * epsilon

z = Lambda(sampling, output_shape=(encoding_dim,))([z_mean, z_log_var])

# Decoder
decoder_inputs = Input(shape=(encoding_dim,))
x_decoded = Dense(16, activation='relu')(decoder_inputs)
x_decoded = Dropout(0.1)(x_decoded)
x_decoded = Dense(32, activation='relu')(x_decoded)
outputs = Dense(input_dim, activation='linear')(x_decoded)

# Custom Reconstruction Loss Layer
class ReconstructionLossLayer(Layer):
    def call(self, inputs):
        inputs, z_decoded = inputs
        diff = inputs - z_decoded
        reconstruction_loss = tf.reduce_mean(tf.square(diff), axis=-1)
        return reconstruction_loss

# Custom KL Loss Layer
class KLLossLayer(Layer):
    def call(self, inputs):
        z_mean, z_log_var = inputs
        kl_loss = -0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
        return kl_loss

# Connect encoder and decoder
decoder = Model(decoder_inputs, outputs)
z_decoded = decoder(z)

# VAE model
vae = Model(inputs, [z_decoded, z_mean, z_log_var])

# Define the custom loss function
reconstruction_loss_layer = ReconstructionLossLayer()
kl_loss_layer = KLLossLayer()

def vae_loss(inputs, z_decoded, z_mean, z_log_var):
    reconstruction_loss = reconstruction_loss_layer([inputs, z_decoded])
    kl_loss_value = kl_loss_layer([z_mean, z_log_var])
    return K.mean(reconstruction_loss + kl_loss_value)

# Compile the VAE
vae.compile(optimizer='adam')

# Custom training loop
epochs = 15
batch_size = 64
train_losses = []
val_losses = []

for epoch in range(epochs):
    epoch_train_loss = 0
    for batch_idx in range(len(X_train_resampled) // batch_size):
        start = batch_idx * batch_size
        end = start + batch_size
        batch_inputs = X_train_resampled[start:end]
        
        with tf.GradientTape() as tape:
            outputs = vae(batch_inputs)
            loss_value = vae_loss(batch_inputs, *outputs)
        
        grads = tape.gradient(loss_value, vae.trainable_weights)
        vae.optimizer.apply_gradients(zip(grads, vae.trainable_weights))
        
        epoch_train_loss += loss_value
    
    epoch_train_loss /= (len(X_train_resampled) // batch_size)
    train_losses.append(epoch_train_loss)
    
    # Evaluate on validation data
    val_loss = vae_loss(X_test, *vae.predict(X_test))
    val_losses.append(val_loss)
    print(f"Epoch {epoch+1}/{epochs}, Training Loss: {epoch_train_loss:.4f}, Validation Loss: {val_loss:.4f}")

# Plot the training and validation losses
plt.plot(range(1, epochs+1), train_losses, label='Training Loss')
plt.plot(range(1, epochs+1), val_losses, label='Validation Loss')
plt.title('VAE Training')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Reconstruct the test data
reconstructed_data = vae.predict(X_test)[0]

# Calculate reconstruction errors
mse = np.mean(np.power(X_test - reconstructed_data, 2), axis=1)

# Threshold optimization
fpr, tpr, thresholds = roc_curve(y_test, mse)
roc_auc = auc(fpr, tpr)
optimal_idx = np.argmax(tpr - fpr)
optimal_threshold = thresholds[optimal_idx]

# Identify potential anomalies
anomalies = X_test[mse > optimal_threshold]


# Ensemble methods
clf1 = LogisticRegression()
clf2 = RandomForestClassifier()
clf3 = SVC(kernel='linear', probability=True)

ensemble_clf = VotingClassifier(estimators=[('lr', clf1), ('rf', clf2), ('svc', clf3)], voting='soft')
ensemble_clf.fit(X_train_resampled, y_train_resampled)

# Evaluate the performance of the ensemble classifier
y_pred_ensemble = ensemble_clf.predict(X_test)

precision = precision_score(y_test, y_pred_ensemble)
recall = recall_score(y_test, y_pred_ensemble)
f1 = f1_score(y_test, y_pred_ensemble)
accuracy = accuracy_score(y_test, y_pred_ensemble)

print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)
print("Accuracy:", accuracy)